# Импорт библиотек и датафрейма

In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
import numpy as np
from scipy.spatial.distance import cdist
import networkx as nx

In [2]:
# отобразим все колонки фрейма данных
pd.set_option('display.max.columns', 48)

In [3]:
df = pd.read_csv("data Модуль Г.csv")
df

,Production Year,Production Date Entered,Operator,County,Town,Field,Producing Formation,Active Oil Wells,Inactive Oil Wells,Active Gas Wells,Inactive Gas Wells,Injection Wells,Disposal Wells,Self-use Well,"Oil Produced, bbl","Gas Produced, Mcf","Water produced, bbl","Taxable Gas, Mcf",Coordinates,Latitude,Longitude
0,1995,1996-12-03,"Buffalo China, Inc.",Erie,BUFFALO,BUFFALO,MEDINA,0,0,1,0,0,0,YES,0,106,0,0,"(42.887691, -78.879374)",42.887691,-78.879374
1,1995,1996-03-29,"Copper Ridge Oil, Inc.",Steuben,WEST UNION,BEECH HILL-INDEPENDENCE,FULMER VALLEY,28,0,0,0,7,0,NO,1229,0,180,0,"(42.094951, -77.725816)",42.094951,-77.725816
2,1994,1995-02-14,"White, Walter W. & Christina L.",Cattaraugus,CARROLLTON,BRADFORD,BRADFORD,3,0,0,0,0,0,NO,462,0,0,0,"(42.023289, -78.628438)",42.023289,-78.628438
3,1994,1995-01-19,"Stiegler, Richard M",Erie,CHEEKTOWAGA,BUFFALO,MEDINA,0,0,1,0,0,0,YES,0,530,0,0,"(42.907071, -78.754318)",42.907071,-78.754318
4,1995,1996-02-12,"Bucher, Charles J",Cattaraugus,ALLEGANY,FIVE MILE,BRADFORD,10,0,0,0,0,0,NO,45,0,80,0,"(42.088061, -78.491258)",42.088061,-78.491258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27921,1995,1996-03-18,"Eastern States Energy, Inc.",Erie,EVANS,BRANT-EDEN,MEDINA,0,0,9,1,0,0,NO,0,35272,710,24002,"(42.638457, -79.027689)",42.638457,-79.027689
27922,1994,1995-03-02,P & G Oil Co.,Allegany,WILLING,FULMER VALLEY,RICHBURG,0,7,0,0,0,0,INACTIVE,0,0,0,0,"(42.085346, -77.922864)",42.085346,-77.922864
27923,1994,1995-02-13,"Vandermark Exploration, Inc.",Steuben,WEST UNION,UNNAMED,ORISKANY,0,0,0,2,0,0,INACTIVE,0,0,0,0,"(42.094951, -77.725816)",42.094951,-77.725816
27924,1995,1996-03-26,Columbia Natural Resources LLC,Chautauqua,GERRY,LAKESHORE,MEDINA,0,0,1,0,0,0,NO,0,3370,0,3370,"(42.194749, -79.250394)",42.194749,-79.250394


# 1. Граф связей построен

In [4]:
# Это потом удаляется

# Количество строк, которые нужно удалить
num_to_remove = 100

# Находим строки с наибольшими значениями в столбце 'Active Oil Wells'
rows_to_remove = df['Active Oil Wells'].nlargest(num_to_remove).index

# Удаляем эти строки из DataFrame
df_cleaned = df.drop(rows_to_remove)

## 1.1 Граф связей построен

In [5]:
df.columns

Index(['Production Year', 'Production Date Entered', 'Operator', 'County',
       'Town', 'Field', 'Producing Formation', 'Active Oil Wells',
       'Inactive Oil Wells', 'Active Gas Wells', 'Inactive Gas Wells',
       'Injection Wells', 'Disposal Wells', 'Self-use Well',
       'Oil Produced, bbl', 'Gas Produced, Mcf', 'Water produced, bbl',
       'Taxable Gas, Mcf', 'Coordinates', 'Latitude', 'Longitude'],
      dtype='object')

In [6]:
# Создание базовой scatter plot
fig = px.scatter_geo(df_cleaned, lat='Latitude', lon='Longitude', color='Active Oil Wells',
                     hover_name='Town', size='Active Oil Wells', size_max=15,
                     title='Earthquakes Around the World')

# Добавление линий между точками в одном County
for county in df_cleaned['County'].unique():
    county_data = df_cleaned[df_cleaned['County'] == county]
    if len(county_data) > 1:
        fig.add_trace(
            go.Scattergeo(
                locationmode='USA-states',
                lon=county_data['Longitude'],
                lat=county_data['Latitude'],
                mode='lines',
                line=dict(width=0.1, color='red'),
                opacity=0.1,
                name=f'{county}',
                showlegend=False
            )
        )
fig.show()


## 1.2 Граф связей даёт возможность понять оптимальные пути транспортировки нефти

Граф связей предоставляет ценные инструменты для анализа оптимальных путей транспортировки нефти, благодаря тому, что каждая точка добычи нефти связана с другой точкой через построенные связи. Эти связи моделируются с использованием алгоритмов, способствующих нахождению кратчайших путей и минимальных затрат на транспортировку. Относительная толщина и интенсивность линий в графе отражает объем транспортировки и частоту перевозок между точками, что помогает визуализировать наиболее интенсивно используемые маршруты. Таким образом, граф не только облегчает поиск оптимальных маршрутов между точками, но и предоставляет инструменты для прогнозирования, анализа узких мест и оптимизации логистических процессов, способствуя эффективному управлению ресурсами и снижению операционных издержек.

## 1.3 Описаны основные метрики полученного графа


1. **Количество точек добычи нефти (Active Oil Wells)**:
   - Эта метрика отображается цветом и размером маркеров на карте. Более крупные и/или более насыщенные цвета могут указывать на большее количество активных скважин в определенных местоположениях.

2. **Распределение по округам (County)**:
   - Граф показывает, как точки добычи нефти распределены по различным округам. Линии между точками в одном округе подчеркивают их принадлежность к одной территориальной единице и могут помочь выявить паттерны в соседстве или близости к точкам добычи.

3. **Интенсивность связей (линий между точками)**:
   - Толщина и интенсивность красных линий между точками внутри одного округа могут отражать частоту перевозок или объем транспортировки нефти между этими точками. Более толстые и насыщенные линии могут указывать на более интенсивные перевозки или более крупные объемы транспортировки.

4. **Соседство и связность**:
   - Граф также позволяет оценить степень соседства и связности точек добычи нефти внутри каждого округа. Это может быть полезно для планирования логистики и оптимизации транспортных маршрутов, учитывая близость и соседство точек.

5. **Визуальная интерпретация данных**:
   - Визуализация позволяет быстро оценить общую картину и паттерны распределения точек добычи нефти на карте мира. Это помогает легче выявить ключевые территории или маршруты, которые требуют особого внимания при планировании транспортировки и управлении нефтяными ресурсами.

Таким образом, полученный граф предоставляет визуальную и аналитическую основу для понимания и оптимизации транспортировки нефти, используя данные о географическом распределении точек добычи и их взаимосвязях внутри территориальных единиц.

# 2. Сетевой анализ данных

## 2.1 Проведен сетевой анализ для определения наиболее важных узлов сети

Определение наиболее выжных узлов

In [7]:
# Создание пустого направленного графа
G = nx.DiGraph()

# Добавление узлов (месторождений)
for index, row in df_cleaned.iterrows():
    G.add_node(row['Town'], latitude=row['Latitude'], longitude=row['Longitude'], active_wells=row['Active Oil Wells'])

# Добавление ребер между узлами (можно на основе расстояний или других метрик)
# Пример добавления ребер по расстоянию между координатами (это пример, реально может потребоваться другое условие для ребер)
for node1, data1 in G.nodes(data=True):
    for node2, data2 in G.nodes(data=True):
        if node1 != node2:
            distance = ((data1['latitude'] - data2['latitude'])**2 + (data1['longitude'] - data2['longitude'])**2)**0.5
            G.add_edge(node1, node2, distance=distance)

# Вычисление метрик центральности
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
closeness_centrality = nx.closeness_centrality(G)

# Вывод наиболее важных узлов сети
most_central_nodes = sorted(G.nodes(), key=lambda x: degree_centrality[x], reverse=True)[:10]
print("Наиболее важные узлы (по степени центральности):")
for i in most_central_nodes:
    print(f"{i}")


Наиболее важные узлы (по степени центральности):
BUFFALO
WEST UNION
CARROLLTON
CHEEKTOWAGA
ALLEGANY
SHERIDAN
ELLERY
LE ROY
WILLING
VILLENOVA


In [8]:
print(df['Town'].unique())


['BUFFALO' 'WEST UNION' 'CARROLLTON' 'CHEEKTOWAGA' 'ALLEGANY' 'SHERIDAN'
 'ELLERY' 'LE ROY' 'WILLING' 'VILLENOVA' 'STAFFORD' 'OTTO' 'ARKWRIGHT'
 'COLDEN' 'ELLINGTON' 'PERRY' 'CHAUTAUQUA' 'SHELDON' 'BENNINGTON'
 'INDEPENDENCE' 'ORCHARD PARK' 'BUSTI' 'HAMBURG' 'EVANS' 'NORTH COLLINS'
 'ELLICOTT' 'PEMBROKE' 'CHARLOTTE' 'LANCASTER' 'TONAWANDA' 'ANDOVER'
 'FAYETTE' 'LEICESTER' 'DARIEN' 'SPRINGPORT' 'CLYMER' 'OWASCO' 'CONCORD'
 'ALMA' 'EAST OTTO' 'JASPER' 'LEON' 'BRANT' 'OLEAN' 'RIPLEY' 'NEWSTEAD'
 'ALDEN' 'BOLIVAR' 'SCIO' 'CAMERON' 'EDEN' 'BATAVIA' 'GENESEE'
 'NORTH HARMONY' 'NUNDA' 'POMFRET' 'PERRYSBURG' 'PHELPS' 'WESTFIELD'
 'SHERMAN' 'HARMONY' 'OWEGO' 'ELMA' 'GERRY' 'CALEDONIA' 'DAYTON' 'YORK'
 'MINA' 'PORTLAND' 'WIRT' 'HANOVER' 'SUMMERHILL' 'WALES' 'HOWARD'
 'MARILLA' 'SANGERFIELD' 'PORTAGE' 'NAPOLI' 'ELBA' 'CHERRY CREEK' 'BATH'
 'LEBANON' 'CARROLL' 'BOSTON' 'COLD SPRING' 'RICHMOND' 'COVINGTON'
 'GENEVA' 'ELLICOTTVILLE' 'CLARENCE' 'STOCKTON' 'HOLLAND' 'GREENWOOD'
 'BARRINGTON' 'AUBURN' 

In [9]:
# Получаем координаты скважин
coordinates = df_cleaned[['Latitude', 'Longitude']].values

# Вычисляем матрицу расстояний между всеми парами скважин
dist_matrix = cdist(coordinates, coordinates, metric='haversine')

# Пример вывода матрицы расстояний (первые 5x5 элементов)
print(dist_matrix[:5, :5])


ValueError: Unknown Distance Metric: haversine

In [ ]:
# pio.write_html(fig, file='earthquakes2.html', auto_open=True)